## Full inference workflwo

The pseudo code below outlines the general steps of the inference pipeline.

In [2]:
#| label: fig-inference
#| echo: true
#| fig-cap: "The WorldCereal inference pipeline"


import  openeo
from    openeo.rest.mlmodel import MlModel
from    openeo.processes import ProcessBuilder

connection = openeo.connect("openeo.dataspace.copernicus.eu")
l2A = connection.load_collection("SENTINEL2_L2A").aggregate_temporal_period(period="month",reducer="mean")  # <1>
sentinel1 = connection.load_collection("SENTINEL1_GRD")
bs = sentinel1.sar_backscatter(coefficient="sigma0-ellipsoid").resample_spatial(resolution=20).aggregate_temporal_period(period="month",reducer="mean")

# <2>

from    openeo import UDF
feature_udf=UDF(code="",runtime="Python") #load UDF to compute presto features based on monthly timeseries
features_cube = l2A.merge_cubes(bs).apply_dimension(dimension='t',process=feature_udf,target_dimension='bands')


model = MlModel.load_ml_model(connection=connection, id="http://myhost.com/my_catboost_stac_metadata.json")

catboost_classifier = lambda data, context: ProcessBuilder.process("predict_catboost",data=data, model=context)
worldcereal_product = features_cube.reduce_dimension(dimension="bands", reducer=catboost_classifier, context=model)

worldcereal_product

ConnectionError: HTTPSConnectionPool(host='openeo.dataspace.copernicus.eu', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x724478182690>: Failed to establish a new connection: [Errno 111] Connection refused'))

1. instead of aggregate_temporal, we'll do more advanced compositing, such as max-NDVI
2. we'll need to add agera5 and dem bands

## Exporting results to workspace

When a product is ready, we want to store it in object storage and adjust stac metadata.



In [1]:
#| label: fig-export
#| echo: true
#| fig-cap: "Workflow steps to export results to object storage"

stac_metadata = worldcereal_product.save_result(format="GTiff")
stac_metadata = connection.datacube_from_process("stac_update",data = stac_metadata) #todo: add custom metadata

connection.datacube_from_process("export_workspace",data = stac_metadata, workspace = "my_workspace", merge="pointer_to_worldcereal_collection")

NameError: name 'worldcereal_product' is not defined